# Vibrations in an elastic surface

Modeling of wave propagation in elastic inhomogeneous media.

In [6]:
import pyglet, random, math, os, sys, numpy as np, copy
from pyglet.window import key, FPSDisplay
from pyglet import image, shapes

# begin coordinat left down corner
window = pyglet.window.Window(840,830,"oscillation field",resizable = False)
batch = pyglet.graphics.Batch()
fps_display = FPSDisplay(window)
fps_display.label = pyglet.text.Label(font_name='Arial', font_size=12, x=20, y=10, color = (0,0,0,255))
pyglet.gl.glClearColor(1,1,1,1)

# ===================== Parallel faber =========================== 
N = 100   # number of elements horizontally and vertically
# Creating a heterogeneous mass environment
PFm  = np.array([[40.+random.random()*200 
                  for x in range(N)]
                  for y in range(N)])
# z coordinates of each of the bodies
PFz  = np.full((N,N),.0)       
# z speed of each body (first derivative of)
PFvz = np.full((N,N),.0)
# z acceleration of each body (2-d derivative of)
PFaz = np.full((N,N),.0)                  

PFx  = np.array([15+i*8 for i in range(N)]) # x coordinates of each of the bodies
PFy  = np.array([25+i*8 for i in range(N)]) # y coordinates of each of the bodies

ON = False;                                 # running the model 
Xm = 0; Ym = 0                              # coordinates mouse click
Mos = 100.                                  # memorize the mass of the oscillator
TS = np.float64(4)                          # time scale
COUNTER = 11
MODE = 1                                    # (0) field setting mode, (1) oscillation mode
# ======================= prepare procedure =============================
Body = [[shapes.Rectangle(PFx[x],PFy[y],8,8,color=(100, 100, 100), batch=batch) for x in range(N)] for y in range(N)]

# field setting mode
Button1 = shapes.Rectangle(30,40,80,30,color=(200, 100, 100), batch=batch); Button1.opacity = 200
labelB1 = pyglet.text.Label('field', font_name='Arial', font_size=14, x=50, y=48, color = (0,0,0,200), batch=batch)
# oscillation mode
Button2 = shapes.Rectangle(120,40,100,30,color=(100, 100, 200), batch=batch); Button2.opacity = 200 
labelB2 = pyglet.text.Label('oscillation', font_name='Arial', font_size=14, x=130, y=48, color = (0,0,0,200), batch=batch)
# clear
Button3 = shapes.Rectangle(230,40,80,30,color=(220, 220, 220), batch=batch); Button3.opacity = 200 
labelB3 = pyglet.text.Label('clear', font_name='Arial', font_size=14, x=250, y=48, color = (0,0,0,200), batch=batch)
# ======================= drawing procedure =============================
@window.event
def on_draw():
    window.clear()
    batch.draw()
    fps_display.draw()
    labelB1.draw(); labelB2.draw(); labelB3.draw()
# ======================= mouse press ============================
@window.event
def on_mouse_press(x, y, button, modifiers):
    global ON, Xm, Ym, MODE, PFm, PFz, PFvz, PFaz, N
    if button == pyglet.window.mouse.LEFT:
        if   x > 30 and x < 110 and y > 40 and y < 70:    # field setting mode
            MODE = 1
        elif x > 120 and x < 220 and y > 40 and y < 70:   # oscillation mode
            MODE = 2; ON = False
        elif x > 230 and x < 310 and y > 40 and y < 70:   # clear
            if MODE == 1: # field setting mode
                PFm  = np.full((N,N),100.)
            else:         # oscillation mode
                PFz  = np.full((N,N),.0)                    # z coordinates of each of the bodies
                PFvz = np.full((N,N),.0)                    # z speed of each body (first derivative of)
                PFaz = np.full((N,N),.0)                    # z acceleration of each body (second derivative of)  
        else:
            if MODE == 1:  # field setting mode
                for xd in range(x-15,x+15):
                    for yd in range(y-15,y+15):
                        Xm = round((xd-25)/8); Ym = round((yd-35)/8)
                        if Xm > 0 and Xm < N-1 and Ym > 0 and Ym < N-1:
                            PFm[Xm,Ym] = 65025 if PFm[Xm,Ym]*1.05 > 65025 else PFm[Xm,Ym]*1.05
            else:          # oscillation mode
                ON = True; Xm = round((x-25)/8); Ym = round((y-35)/8)

@window.event                
def on_mouse_drag(x, y, dx, dy, buttons, modifiers):
    global ON, Xm, Ym, MODE, PFm
    if MODE == 1:  # field setting mode
        for xd in range(x-15,x+15):
            for yd in range(y-15,y+15):
                Xm = round((xd-25)/8); Ym = round((yd-35)/8)
                if Xm > 0 and Xm < N-1 and Ym > 0 and Ym < N-1:
                    PFm[Xm,Ym] = 65025 if PFm[Xm,Ym]*1.05 > 65025 else PFm[Xm,Ym]*1.05
# ======================= update procedure ============================
def update(dt):  
    global N,Body, ON, PFz, PFvz, PFaz, PFx, PFy, PFm, COUNTER, label, Xm, Ym, Mos, MODE

    if MODE == 1: #==== MODE 1 =========================== 
        for y in range(N-2):
            for x in range(N-2):
                cd = 255-PFm[x,y]**.5
                Body[y+1][x+1].color = (cd,cd,cd)
    else: #============ MODE 2 ===========================
        if ON:                                                   # excitation
            PFvz[Xm,Ym] = 10
            Mos = PFm[Xm,Ym]; PFm[Xm,Ym] = 1000.
            ON = False
            COUNTER = 0
        if COUNTER == 10:
            PFvz[Xm,Ym] = -9
            PFm[Xm,Ym] = Mos

        for y in range(N-2):                                    # without edge operations
            for x in range(N-2):
                cen = PFz[x+1,y+1]
                c = cen*1000
                if c >= 0:
                    cd = 255-c**.5
                    Body[y+1][x+1].color = (255,cd,cd)
                else:
                    cd = 255-(-c)**.5
                    Body[y+1][x+1].color = (cd,cd,255)

                PFaz[x+1,y+1] = (PFz[x+1,y]-cen + PFz[x+1,y+2]-cen + PFz[x,y+1]-cen + PFz[x+2,y+1]-cen + \
                                 PFz[x,y]-cen   + PFz[x+2,y+2]-cen + PFz[x,y+2]-cen + PFz[x+2,y]-cen - \
                                 PFvz[x+1,y+1]) / PFm[x+1,y+1]
                PFvz[x+1,y+1] += PFaz[x+1,y+1]*TS

        PFz  += PFvz*TS  
        
        # ============ MODE 2 ===========================
        COUNTER += 1    
# =========================== MAIN LOOP ============================
if __name__ == "__main__":
    pyglet.clock.schedule_interval(update, 1/60)
    pyglet.app.run()
    pyglet.clock.unschedule(update)   # necessary to restart the countdown of time intervals
#========================================================================